In [24]:
from robot_library_py import *
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Joy, JointState
from intera_core_msgs.msg import JointCommand

from threading import Thread
import time
import numpy as np 
from std_msgs.msg import Bool
import rotm2euler

from utils_camera import MyRealSense, CVCamera
import cv2
from matplotlib import pyplot as plt

In [2]:
!v4l2-ctl --list-devices

Streaming Camera W2G: Streaming (usb-0000:00:14.0-1.1):
	/dev/video7
	/dev/video10
	/dev/media2

Intel(R) RealSense(TM) Depth Ca (usb-0000:00:14.0-2):
	/dev/video0
	/dev/video1
	/dev/video2
	/dev/video3
	/dev/video8
	/dev/video9
	/dev/media0
	/dev/media1



In [3]:
cam_wrist=MyRealSense()
cam_front=CVCamera(7)

In [4]:
image_wrist=cam_wrist.get_current_frame()  
image_front=cam_front.get_current_frame()     

image=np.concatenate([image_wrist, image_front], axis=1)
image_wrist.shape, image_front.shape, image.shape

((240, 320, 3), (240, 320, 3), (240, 640, 3))

In [5]:
class ROSInterface:
    def __init__(self, node, robot):
        
        self.robot = robot
        self.node = node
        self.jointMap = {name: ind for ind, name in enumerate(self.robot.jointNames)} 

        self.latest_joint_states = None
        self.latest_gripper_state=False

    def joint_states_callback(self, msg):
        self.latest_joint_states = msg
        q = self.robot.getJoints()
        # print('joint_states:', q)
        for ind, name in enumerate(msg.name):
            if name in self.jointMap:
                q[self.jointMap[name]] = msg.position[ind]
        self.robot.setJoints(q)
    
    def gripper_callback(self, msg):
        self.latest_gripper_state=msg.data

    def spin_thread(self):
        self.node.create_subscription(JointState, 'robot/joint_states', self.joint_states_callback, 10) 
        self.node.create_subscription(Bool, '/gripper_command', self.gripper_callback, 10)
        rclpy.spin(self.node)

In [6]:
rclpy.init()
robot = URDFModel('/home/ns/sawyer_robot_ros2/src/teleop_script/sawyer.urdf')
jointMap = {name: ind for ind, name in enumerate(robot.jointNames)}

node = Node('pub_dbg')
q = robot.getJoints()
ros_interface = ROSInterface(node, robot)
cmd_pub = node.create_publisher(JointCommand, '/robot/limb/right/joint_command', 10)
pass 

sawyer
link: 0
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: base
link: 1
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name: torso
link: 2
	cog:    0    0 -0.5
	inertia:    5.06359    6.08689    4.96192 0.00105311   0.801996 0.00103417
	mass: 60.864
	name: pedestal
link: 3
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: controller_box
link: 4
	cog: 0 0 0
	inertia: 1 1 1 0 0 0
	mass: 1
	name: pedestal_feet
link: 5
	cog:  -0.0006241 -2.8025e-05    0.065404
	inertia:   0.0067599   0.0067877   0.0074031  1.5888e-05 -6.1904e-07 -4.2024e-05
	mass: 2.0687
	name: right_arm_base_link
link: 6
	cog: 0.024366 0.010969  0.14363
	inertia:  0.053314  0.057902  0.023659 0.0080179  0.011734 0.0047093
	mass: 5.3213
	name: right_l0
link: 7
	cog:   0.0053207 -2.6549e-05      0.1021
	inertia:    0.011833   0.0082709   0.0049661  4.2124e-07  4.9425e-05 -4.4669e-06
	mass: 1.5795
	name: head
link: 8
	cog: 0 0 0
	inertia: 1e-08 1e-08 1e-08     0     0     0
	mass: 0.0001
	name

In [7]:
# ros_interface.spin_thread()
t1 = Thread(target=ros_interface.spin_thread)
t1.start()

In [11]:
msg=ros_interface.latest_joint_states
poss_prev=np.array(msg.position)

msgs=[]
imgs=[]
motion_detected=False
stop_count=0

st=time.time()
for i in range(1000000):
    # time.sleep(0.1)

    image_wrist=cam_wrist.get_current_frame()  
    image_front=cam_front.get_current_frame() 


    msg=ros_interface.latest_joint_states
    msgs.append(msg)

    imgs.append( (image_wrist, image_front) )



    poss=np.array(msg.position)
    d=np.linalg.norm(poss-poss_prev)
    if np.allclose(poss_prev, poss):  # no motion
        if motion_detected:
            # print('motion stopped, d=', d)
            stop_count+=1
            if stop_count>1:
                print('stop')
                break
    elif not motion_detected:
        print('motion detected')
        motion_detected=True
        
    poss_prev=poss.copy()

    image=np.concatenate([image_wrist, image_front], axis=1)
     

    cv2.imshow('frame',image)
    if cv2.waitKey(1) == 27: 
        print('UI closed')
        break  # esc to quit
cv2.destroyAllWindows() 



print('done')
dt=time.time()-st
len(msgs), dt, len(msgs)/dt

motion detected
stop
done


(261, 8.668341636657715, 30.109565467084515)

In [12]:
len(imgs), len(msgs)

(206, 206)

In [12]:
msg=ros_interface.latest_joint_states
poss_prev=np.array(msg.position)

msgs=[]
imgs=[]
grips=[]
motion_detected=False
stop_count=0

started=False  #make sure gripper is closed before starting

# st=time.time()
for i in range(1000000):
    image_wrist=cam_wrist.get_current_frame()  
    image_front=cam_front.get_current_frame() 
    msg=ros_interface.latest_joint_states
    gripper_isopen=ros_interface.latest_gripper_state

    if len(msg.position)==1:
        continue
 
    if not started and gripper_isopen:
        print('started')
        started=True
        st=time.time()

    if started:
        msgs.append(msg)
        imgs.append( (image_wrist, image_front) ) 
        grips.append(gripper_isopen)
    

    image=np.concatenate([image_wrist, image_front], axis=1) 
    cv2.imshow('frame',image)
    if cv2.waitKey(1) == 27: 
        print('UI closed')
        break  # esc to quit
cv2.destroyAllWindows() 


print('done')
dt=time.time()-st
len(msgs), dt, len(msgs)/dt

started
UI closed
done


(790, 29.70841121673584, 26.591795644560218)

In [11]:
# cam_front.close()
# cam_wrist.close()

In [13]:
len(imgs)

790

In [33]:
j=0
for img in imgs:
    image_wrist, image_front=img
    image=np.concatenate([image_wrist, image_front], axis=1)
    cv2.imshow('frame',image)
    if cv2.waitKey(1) == 27: 
        print('UI closed')
        break  # esc to quit
    time.sleep(1/30)

    j=j+1
    if j>650:
        break

cv2.destroyAllWindows() 

In [20]:
image.shape

(240, 640, 3)

In [22]:
cv2.destroyAllWindows()

In [30]:
# plt.imshow(image)

In [31]:
len(imgs)

790

In [34]:
tosave={}

N=650
tosave['msgs']=msgs[:N]
tosave['imgs']=imgs[:N]
tosave['grips']=grips[:N]

import pickle
with open('sawyer_spoon_1_sep9.pkl', 'wb') as f:
    pickle.dump(tosave, f)

